In [3]:
# Cell 1 imports and setup
import numpy as np
from numpy import pi
import time 
from collections import Counter

from braket.circuits import Circuit
from braket.devices import LocalSimulator
from braket.aws import AwsDevice

In [4]:
# Cell 2: Helper Function

def hellinger_fidelity(counts1, counts2):
    """
    Calculate Hellinger fidelity between two probability distributions
    """
    # Get all possible outcomes
    all_outcomes = set(counts1.keys()) | set(counts2.keys())

    # Convert counts to probabilities
    total1 = sum(counts1.values())
    total2 = sum(counts2.values())

    prob1 = {outcome: counts1.get(outcome, 0) / total1 for outcome in all_outcomes}
    prob2 = {outcome: counts2.get(outcome, 0) / total2 for outcome in all_outcomes}

    # Calculate Hellinger fidelity
    fidelity = sum(np.sqrt(prob1[outcome] * prob2[outcome]) for outcome in all_outcomes)
    return fidelity ** 2

In [5]:
# Cell 3: Circuit definitions (translated from the original Qiskit code)
circuits_data = {
    "P0+": [["id", 0], ["id", 1], ["id", 2]],
    "P0-": [["cx", 2, 1], ["x", 0], ["x", 1], ["x", 2], ["ccx", 2, 1, 0], ["x", 1], ["x", 2]],
    "P1+": [["ccx", 2, 1, 0], ["cx", 2, 1], ["x", 2]],
    "P1-": [["cx", 1, 0], ["x", 2]],
    "P2+": [["cx", 1, 0], ["x", 1]],
    "P2-": [["ccx", 2, 1, 0], ["cx", 2, 1], ["x", 1]],
    "P3+": [["cx", 2, 1], ["x", 0], ["x", 1], ["x", 2], ["ccx", 2, 1, 0]],
    "P3-": [["x", 1]],
    "P4+": [["x", 2]],
    "P4-": [["cx", 2, 1], ["x", 0], ["x", 1], ["x", 2], ["ccx", 2, 1, 0], ["x", 2], ["x", 1]],
    "P5+": [["ccx", 2, 1, 0], ["cx", 2, 1], ["x", 0], ["x", 2]],
    "P5-": [["x", 1], ["x", 2], ["cx", 1, 0], ["x", 1]],
    "P6+": [["x", 1], ["cx", 1, 0]],
    "P6-": [["ccx", 0, 1, 2], ["x", 1], ["x", 0]],
    "P7+": [["cx", 2, 1], ["x", 2], ["x", 1], ["ccx", 2, 1, 0]],
    "P7-": [["x", 2], ["x", 1], ["x", 0]],
}

In [6]:
# Cell 4: Auxiliary Circuit Creation Function
def create_aux_circuit(operations):
    """
    Create auxiliary circuit with specified operations
    """
    circuit = Circuit()
    for op in operations:
        if op[0] == "id":
            # Identity gate (no-op in Braket)
            continue
        elif op[0] == "x":
            circuit.x(op[1])
        elif op[0] == "h":
            circuit.h(op[1])
        elif op[0] == "cx":
            circuit.cnot(op[1], op[2])
        elif op[0] == "ccx":
            circuit.ccnot(op[1], op[2], op[3])
        elif op[0] == "z":
            circuit.z(op[1])
        elif op[0] == "s":
            circuit.s(op[1])
    return circuit

In [7]:
# Cell 5: Main Circuit Creation Function
def create_main_circuit(aux_circuit):
    """
    Create main quantum circuit with auxiliary circuit
    """
    circuit = Circuit()

    # Apply initial gates (equivalent to original QC function)
    circuit.h(0)
    circuit.h(1)
    circuit.h(2)
    circuit.s(1)
    circuit.z(0)
    circuit.phaseshift(2, pi/2) # P gate equivalent 

    # Add auxiliray circuit
    circuit.add(aux_circuit)

    # Apply final gates
    circuit.phaseshift(1, pi/2) # P gate equivalent
    circuit.h(1)

    return circuit

In [8]:
# Cell 6: Create all circuits
main_circuits = []
aux_circuits = []

for name, ops in circuits_data.items():
    aux_circuit = create_aux_circuit(ops)
    main_circuit = create_main_circuit(aux_circuit)

    aux_circuits.append((name, aux_circuit))
    main_circuits.append((name, main_circuit))

print(f"Created {len(main_circuits)} circuits")

# Example: Visualize one circuit (P6-)
example_circuit = main_circuits[13][1] # P6- circuit
print(f"Example circuit (P6-) depth: {example_circuit.depth}")
print(f"Example circuit gate count: {len(example_circuit.instructions)}")

Created 16 circuits
Example circuit (P6-) depth: 6
Example circuit gate count: 11


In [9]:
# Cell 7: Device Selection and Execution
# Choose device (LocalSimulator for free tier, or AWS device for real wuantum hardware)
# For free tier
device = LocalSimulator()
print("Using LocalSimulator (free tier)")

# For AWS quantum simulators (uses free tier hour)
# device = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/sv1")

# For real quantum hardware (costs money):
# device = AwsDevice("arn:aws:braket:us-east-1::device/qpu/ionq/ionQdevice")

print(f"Running {len(main_circuits)} circuits...")

Using LocalSimulator (free tier)
Running 16 circuits...


In [11]:
# Cell 8: Run experiments and calculate fidelities
from braket.circuits.result_types import Probability

fidelities = []
start_time = time.time()

for i, (name, circuit) in enumerate(main_circuits):
    print(f"Running circuit {i}: {name}")

    # Add measurement to circuit for this run
    measured_circuit = circuit.copy()
    measured_circuit.add_result_type(Probability(target = [1]))

    # Run circuit twice for fidelity comparison
    task1 = device.run(measured_circuit, shots = 1024)
    task2 = device.run(measured_circuit, shots = 1024)

    # Get results
    result1 = task1.result()
    result2 = task2.result()

    # Convert probabilities to counts for fidelity calculation
    probs1 = result1.values[0] # [prob_0, prob_1]
    probs2 = result2.values[0]

    # Extract counts
    counts1 = {'0': int(probs1[0] * 1024), '1': int(probs1[1] * 1024)}
    counts2 = {'0': int(probs2[0] * 1024), '1': int(probs2[1] * 1024)}

    # Calculate fidelity
    fidelity = hellinger_fidelity(counts1, counts2)
    fidelities.append(fidelity)

    print(f" Fidelity: {fidelity:.6f}")

execution_time = time.time() - start_time
print(f"\nTotal execution time: {execution_time:.2f} seconds")

Running circuit 0: P0+
 Fidelity: 1.000000
Running circuit 1: P0-
 Fidelity: 1.000000
Running circuit 2: P1+
 Fidelity: 1.000000
Running circuit 3: P1-
 Fidelity: 1.000000
Running circuit 4: P2+
 Fidelity: 1.000000
Running circuit 5: P2-
 Fidelity: 1.000000
Running circuit 6: P3+
 Fidelity: 1.000000
Running circuit 7: P3-
 Fidelity: 1.000000
Running circuit 8: P4+
 Fidelity: 1.000000
Running circuit 9: P4-
 Fidelity: 1.000000
Running circuit 10: P5+
 Fidelity: 1.000000
Running circuit 11: P5-
 Fidelity: 1.000000
Running circuit 12: P6+
 Fidelity: 1.000000
Running circuit 13: P6-
 Fidelity: 0.999157
Running circuit 14: P7+
 Fidelity: 1.000000
Running circuit 15: P7-
 Fidelity: 1.000000

Total execution time: 0.30 seconds


In [12]:
# Cell 9: Results Analysis
print("\n" + "="*50)
print("FIDELITY SUMMARY")
print("="*50)

for i, (name, fidelity) in enumerate(zip([name for name, _ in main_circuits], fidelities)):
    print(f"Circuit {i:2d} ({name:3s}): {fidelity:.6f}")

print("\n" + "-"*30)
print(f"Average fidelity: {np.mean(fidelities):.6f}")
print(f"Min fidelity: {np.min(fidelities):.6f}")
print(f"Max fidelity: {np.max(fidelities):.6f}")
print(f"Std deviation: {np.std(fidelities):.6f}")

# Optional: Compare with original results if available
print(f"\nExecution completed sucessfully")
print(f"All {len(fidelities)} circutits executed on Amazon Braket")


FIDELITY SUMMARY
Circuit  0 (P0+): 1.000000
Circuit  1 (P0-): 1.000000
Circuit  2 (P1+): 1.000000
Circuit  3 (P1-): 1.000000
Circuit  4 (P2+): 1.000000
Circuit  5 (P2-): 1.000000
Circuit  6 (P3+): 1.000000
Circuit  7 (P3-): 1.000000
Circuit  8 (P4+): 1.000000
Circuit  9 (P4-): 1.000000
Circuit 10 (P5+): 1.000000
Circuit 11 (P5-): 1.000000
Circuit 12 (P6+): 1.000000
Circuit 13 (P6-): 0.999157
Circuit 14 (P7+): 1.000000
Circuit 15 (P7-): 1.000000

------------------------------
Average fidelity: 0.999947
Min fidelity: 0.999157
Max fidelity: 1.000000
Std deviation: 0.000204

Execution completed sucessfully
All 16 circutits executed on Amazon Braket
